In [1]:
from backend.data_model.time_series import DataMultibandTimeSeries
from backend.data_model.time_series import TimeSeriesMongoDataBase
from backend.offline.offline_algorithms import Birch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import string

In [2]:
def extract_feature_matrix(database, id_list):
    time_series_list = database.get_many('macho', id_list)
    feature_vectors = []
    for time_series in time_series_list:
        feature_vector = time_series.feature_vector
        if len(feature_vector) != 0:
            feature_vectors.append(feature_vector)
    return np.matrix(np.vstack(feature_vectors))

In [3]:
def plot_cluster_list(centers, clusters, database):
    plt.plot(centers[:, 0], centers[:, 1], 'x')
    colors = plt.cm.Spectral(np.linspace(0, 1, len(clusters)))
    for cluster_indices, col in zip(clusters, colors):
        cluster_data = extract_feature_matrix(database, id_list)
        plt.plot(cluster_data[:, 0], cluster_data[:, 1], 'o', markerfacecolor=col)
    plt.show()

In [4]:
mongodb = TimeSeriesMongoDataBase('lightcurves')
lightcurves = mongodb.find_many('macho', {})

In [5]:
threshold = 2
birch = Birch(threshold, 'd1', 'r', 4)

In [6]:
birch.add_many_time_series(lightcurves)

TypeError: unsupported operand type(s) for *: 'TimeSeriesBand' and 'TimeSeriesBand'

In [ ]:
local_centers, local_clusters = birch.get_cluster_list(mode='local')
plot_cluster_list(local_centers, local_clusters, mongodb)

In [ ]:
global_centers, global_clusters = birch.get_cluster_list(mode='global')
plot_cluster_list(global_centers, global_clusters, mongodb)